In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KDTree
#from scores import scores
import time
import warnings
warnings.filterwarnings("ignore")

### 4.) Import Data

In [2]:
%run ../notebooks/utils.ipynb

In [3]:
trainandknn_Xy_original_df = pd.read_csv("../data/train_new.csv", sep="|")
train_Xy_original_df, knn_Xy_original_df = train_test_split(trainandknn_Xy_original_df,train_size=0.75, random_state=42) # if FINAL_SUBMISSION else 0.8) #small
test_X_original_df  = pd.read_csv("../data/test.csv", sep="|").iloc[0:500] #TODO: For faster testing we use less data from the test set
test_final_X_df = pd.read_csv("../data/test.csv", sep="|")

train_Xy_wo_knn_df = pd.read_csv("../data/train_new.csv", sep="|")

#Only for test routines
val_Xy_original_df = pd.read_csv("../data/val_new.csv", sep="|")
train_complete_Xy_original_df = pd.read_csv("../data/train.csv", sep="|")

In [18]:
#convention for variables names: datasetname_columntype_transformstatus_dataframeornot
train_y_original_df = train_Xy_original_df[["fraud"]].copy()
train_X_original_df = train_Xy_original_df.copy().drop("fraud", axis=1)

knn_y_original_df = knn_Xy_original_df[["fraud"]].copy()
knn_X_original_df = knn_Xy_original_df.copy().drop("fraud", axis=1)

# Only for test routie#nes
val_y_originial_df = val_Xy_original_df[["fraud"]].copy()
val_X_originial_df = val_Xy_original_df.copy().drop("fraud", axis=1)

train_Xy_wo_knn_df = train_Xy_wo_knn_df.drop(train_Xy_wo_knn_df[train_Xy_wo_knn_df.trustLevel >= 3].index)

train_y_wo_knn_df = train_Xy_wo_knn_df[["fraud"]].copy()
train_X_wo_knn_df = train_Xy_wo_knn_df.copy().drop("fraud", axis=1)

train_complete_y_originial_df = train_complete_Xy_original_df[["fraud"]].copy()
train_complete_X_originial_df = train_complete_Xy_original_df.copy().drop("fraud", axis=1)

In [5]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
transformer = DataTransformer(scaler)

# Adding new Features to train and test set
train_X_unscaled_df = transformer.add_features(train_X_original_df)
test_X_unscaled_df = transformer.add_features(test_X_original_df)
knn_X_unscaled_df = transformer.add_features(knn_X_original_df)

val_X_unscaled_df = transformer.add_features(val_X_originial_df)
train_complete_X_unscaled_df = transformer.add_features(train_complete_X_originial_df) 

transformer.fit_scaler(transformer.add_features(train_complete_X_unscaled_df.append(test_X_unscaled_df, sort=False)))
train_X_scaled_df = transformer.apply_scaler(train_X_unscaled_df)
knn_X_scaled_df   = transformer.apply_scaler(knn_X_unscaled_df)


test_X_scaled_df  = transformer.apply_scaler(test_X_unscaled_df)
val_X_scaled_df = transformer.apply_scaler(val_X_unscaled_df)
train_complete_X_scaled_df = transformer.apply_scaler(train_complete_X_unscaled_df)

train_X_wo_knn_unscaled_df = transformer.add_features(train_X_wo_knn_df.copy())
train_X_wo_knn_scaled_df = transformer.apply_scaler(train_X_wo_knn_unscaled_df)

# labels
train_y_df = train_y_original_df.copy()
val_y_df = val_y_originial_df.copy()
knn_y_df = knn_y_original_df.copy()

train_complete_X_scaled_df = transformer.apply_scaler(transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud'])))
train_complete_y_df = train_complete_Xy_original_df.copy().fraud
train_complete_X_unscaled_df = transformer.add_features(train_complete_Xy_original_df.copy().drop(columns=['fraud']))

test_final_X_df = transformer.add_features(test_final_X_df)


train_complete_y_df = train_complete_y_df.drop(train_complete_X_unscaled_df[train_complete_X_unscaled_df.trustLevel >= 3].index)
train_complete_X_unscaled_df = train_complete_X_unscaled_df.drop(train_complete_X_unscaled_df[train_complete_X_unscaled_df.trustLevel >= 3].index)

display(train_complete_X_unscaled_df.head(5))
display(train_complete_y_df.head(5))

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems
5,1,770,11.09,11,5,2,0.033766,0.014403,0.423077,26.0
7,2,1545,22.80,0,8,4,0.006472,0.014757,0.000000,10.0
9,2,725,41.08,10,2,4,0.037241,0.056662,0.370370,27.0
15,1,870,32.45,3,1,5,0.006897,0.037299,0.500000,6.0
23,2,125,25.50,5,6,2,0.192000,0.204000,0.208333,24.0


5     1
7     0
9     0
15    0
23    0
Name: fraud, dtype: int64

## Scoring functions
Defining multiple scores which should be tracked in the HyperParamSearch Object

In [6]:
#score = scores.Scores()

In [7]:
scoring = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall'}#, "AP": score.average_precision, "DMC" : score.dmc_score}

## Defining the paramteres which should be tuned
To tune the hyperparameters, i looked into the [documentation here](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster). `randint` can be used for integer values, for float values, use `uniform`. 

You can also use a Grid search on single parameters to get a feeling for a good interval. If you want to try only two possibilities, you can create a list like for the `scale_pos_weight` parameter.

**Note: For the classifiers which work without gpu support, you can probably set a parameter n_jobs=-1 to use all processors**

In [8]:
params = {
    "max_depth": randint(2, 6),  # default 3
    "n_estimators": randint(200, 450),  # default 100
    "learnin_rate" :  uniform(0.1, 0.7),
    "gamma" : uniform(0,2),
    "min_child_weight" : uniform(0,50),
    "max_delta_step" : uniform(0,10), #Set it to value of 1-10 might help control the update.
    "reg_lambda" : uniform(0.1,2),
}

## Creating a classifier with some default values
Not all paramters of a classifier should be fine tuned. For SVM for example, the `kernel`-paramter should be set manually. In the case of xgboost, some things like the objective, the booster and the tree method should not be tuned. The choice of paramters depend on the specific classifier

In [9]:
from xgboost import XGBClassifier
default_xgb = XGBClassifier(booster="gbtree",tree_method='gpu_hist', disable_default_eval_metric=1,objective='binary:logistic',eval_metric='aucpr', n_gpus=1, verbosity=2)
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=500,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='DMC')

In [10]:
search = RandomizedSearchCV(default_xgb, scoring=scoring, param_distributions=params, random_state=42, n_iter=10000,
                            cv=3, verbose=1, n_jobs=-1, return_train_score=True,refit='AUC')
search.fit(train_complete_X_unscaled_df, train_complete_y_df)
results = search.cv_results_

Fitting 3 folds for each of 10000 candidates, totalling 30000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   40.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  9.4min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 11.8min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 14.6min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 17.5min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 20.7min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 27.9min
[Parallel(n_jobs=-1)]: Done 11234 tasks      |

In [12]:
search.best_params_

{'gamma': 0.5361394269637965,
 'learnin_rate': 0.5259503267995072,
 'max_delta_step': 2.809035999095031,
 'max_depth': 2,
 'min_child_weight': 0.9672585190973582,
 'n_estimators': 403,
 'reg_lambda': 0.16439595189061076}

In [15]:
train_complete_Xy_scaled_df = train_complete_X_scaled_df.copy()
train_complete_Xy_scaled_df['fraud'] =train_complete_y_df.copy()

train_Xy_wo_knn_scaled_df = train_X_wo_knn_scaled_df.copy()
train_Xy_wo_knn_scaled_df['fraud'] = train_y_wo_knn_df

val_Xy_scaled = val_X_scaled_df.copy()
val_Xy_scaled['fraud'] = val_y_df.copy()

# need to cut trustLevel >= 3

In [17]:
test_inverse = transformer.inverse_scale(train_Xy_wo_knn_scaled_df.copy())
test_inverse.head()

ValueError: operands could not be broadcast together with shapes (1503,11) (10,) (1503,11) 

In [19]:
scorings = {'AUC': 'roc_auc', 'FBeta': metrics.make_scorer(metrics.fbeta_score, beta=0.5172), "Precision":'precision', "Recall": 'recall'}#, "AP": score.average_precision, "DMC" : score.dmc_score}
xgbo = search.best_estimator_
result_dict = test_classification(xgbo,df_train=train_Xy_wo_knn_scaled_df, df_val=val_Xy_scaled)

Results Fix Split: 
DMC Score: 15  ---  Normalized DMC Score: 0.0398936170212766, 

Results Cross Validation: 
DMC Score: 49.0  ---  Normalized DMC Score: 0.1303191489361702 


In [16]:
#dmc = np.mean(search.cv_results_['mean_test_DMC'])
#ap = np.mean(search.cv_results_['mean_test_AP'])
#precision = np.mean(search.cv_results_['mean_test_Precision'])
#recall = np.mean(search.cv_results_['mean_test_Recall'])

In [17]:
#print("DMC: {} --- Average Precision: {} --- Precision: {} --- Recall: {}".format(dmc, ap, precision, recall))

DMC: -87.93675199574243 --- Average Precision: 0.844044914297449 --- Precision: 0.5144370515276927 --- Recall: 0.3709619890787609


In [13]:
search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, disable_default_eval_metric=1,
       eval_metric='aucpr', gamma=0.5361394269637965,
       learnin_rate=0.5259503267995072, learning_rate=0.1,
       max_delta_step=2.809035999095031, max_depth=2,
       min_child_weight=0.9672585190973582, missing=None, n_estimators=403,
       n_gpus=1, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=0.16439595189061076,
       scale_pos_weight=1, seed=None, silent=True, subsample=1,
       tree_method='gpu_hist', verbosity=2)

In [14]:
search.best_params_

{'gamma': 0.5361394269637965,
 'learnin_rate': 0.5259503267995072,
 'max_delta_step': 2.809035999095031,
 'max_depth': 2,
 'min_child_weight': 0.9672585190973582,
 'n_estimators': 403,
 'reg_lambda': 0.16439595189061076}